## Span Identification
"model_name": "google-bert/bert-base-multilingual-uncased",
"max_seq_length": 128,
"train_batch_size": 8,
"eval_batch_size": 8,
"num_train_epochs": 1

In [77]:
import os
import glob
import codecs
import csv
import pandas as pd
import math
import numpy as np
from tqdm import tqdm, tqdm_notebook, trange
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from transformers import (BertConfig, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup)

### Config

In [ ]:
train_articles = "datasets/train"
dev_articles = "datasets/dev"
train_SI_labels = "datasets/train_SI_labels"
dev_SI_labels = "datasets/dev_SI_labels"

In [ ]:
device = torch.device("cuda")
n_gpu = torch.cuda.device_count()

In [2]:
MODEL_CLASSES = {"bert": (BertConfig, BertForSequenceClassification, BertTokenizer)}
args = {"data_dir": "bert/",
        "model_type": "bert",
        "model_name": "google-bert/bert-base-multilingual-uncased",
        "output_dir": "output_dir/output_model_SI",
        "max_seq_length": 128,
        "train_batch_size": 8,
        "eval_batch_size": 8,
        "num_train_epochs": 1,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "gradient_accumulation_steps": 1,
        "save_steps": 2000}

#### Data pre-processing

In [3]:
def merge_overlapping(indices_list):
  """
  Merges overlapping indices and sorts indices from list of tuples
  """
  # If no propaganda, return empty list
  if indices_list == []:
    return []

  # Sort the list
  indices_list = sorted(indices_list)
  i = 0

  # Going through tuples from the beginning, see if it overlaps with the nex one 
  # and merge it if so
  while True:
    if i == len(indices_list) - 1:
      break
    
    # If the next one is within the range of current, just delete next one. If
    # overlapping, then merge the range
    elif indices_list[i][1] >= indices_list[i + 1][0]:
      if indices_list[i][1] >= indices_list[i + 1][1]:
        pass
      else:
        indices_list[i] = (indices_list[i][0], indices_list[i+1][1])
      indices_list.pop(i+1)

    # Go to next element if not overlapping
    else:
      i += 1

  return indices_list

In [4]:
def article_to_sequences(article_id, article, tokenizer):
  """
  Divides article into sequences, dividing first by sentences then to powersets
  of the sentences
  """
  # Split the lines by sentences
  curr = 0
  lines = article.split("\n")
  sequences = []
  seq_starts = []
  seq_ends = []

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue
    if line == "":
      curr += 1
      continue

    # Tokenize the line
    tokenized = tokenizer.tokenize(line)

    # For each token, do:
    seq_start = 0
    for ind, token in enumerate(tokenized):
      # Get the token without ## sign
      mod_start_token = token.replace("##", "")

      # Find the start of the sequence in line
      seq_start = line.lower().find(mod_start_token, seq_start)

      # Update the end of the sequence
      seq_end = seq_start

      # For each following tokens in the line, do
      for iter in range(1, len(tokenized) + 1 - ind):
        # Also modify this token
        mod_end_token = tokenized[ind + iter - 1].replace("##", "")
        # Find the end of the token
        seq_end = line.lower().find(mod_end_token, seq_end) + len(mod_end_token)

        sequences.append(tokenizer.convert_tokens_to_string(tokenized[ind: ind + iter]))
        seq_starts.append(curr + seq_start)
        seq_ends.append(curr + seq_end)

      # Update the start of the sequence
      seq_start += len(mod_start_token)

    # Update the current whereabouts
    curr += len(line) + 1

  dataframe = pd.DataFrame(None, range(len(sequences)), ["id", "seq_starts", "seq_ends", "label", "text"])
  dataframe["id"] = [article_id] * len(sequences)
  dataframe["seq_starts"] = seq_starts
  dataframe["seq_ends"] = seq_ends
  dataframe["label"] = [0] * len(sequences)
  dataframe["text"] = sequences
  return dataframe


In [5]:
def article_labels_to_sequences(article, indices_list):
  """
  Divides article into sequences, where each are tagged to be propaganda or not
  """
  # Start at 0 indices, and split the article into lines
  curr = 0
  lines = article.split("\n")
  sequences = {}

  # For each lines, do:
  for line in lines:
    # If an empty line, just continue after adding \n character
    if line == "":
      curr += 1
      continue

    # If nothing in indices_list or current line is not part of propaganda, 
    # just mark it to be none 
    elif indices_list == [] or curr + len(line) <= indices_list[0][0]:
      sequences[line] = 0

    # If current line is part of propaganda, do:
    else:
      # If the propaganda is contained within the line, add it accordingly
      # and pop that indices range
      if curr + len(line) >= indices_list[0][1]:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:indices_list[0][1] - curr]] = 1
        sequences[line[indices_list[0][1] - curr:]] = 0
        indices_list.pop(0)
      # If the propaganda goes over to the next line, add accordingly and 
      # modify that indices range
      else:
        sequences[line[:indices_list[0][0] - curr]] = 0
        sequences[line[indices_list[0][0] - curr:]] = 1
        indices_list[0] = (curr + len(line) + 2, indices_list[0][1])  
        indices_list[0] = tuple(indices_list[0])                       
    # Add the current line length plus \n character
    curr += len(line) + 1

  dataframe = pd.DataFrame(None, range(len(sequences)), ["label", "text"])
  dataframe["label"] = sequences.values()  
  dataframe["text"] = sequences.keys()
  return dataframe

In [6]:
def articles_to_dataframe(article_folder, label_folder):
  """
  Preprocesses the articles into dataframes with sequences with binary tags
  """
  # First sort the filenames and make sure we have label file for each articles
  article_filenames = sorted(glob.glob(os.path.join(article_folder, "*.txt")))
  label_filenames = sorted(glob.glob(os.path.join(label_folder, "*.labels.tsv"))) 
  assert len(article_filenames) == len(label_filenames)

  # Initialize sequences
  sequences = []

  # For each article, do:
  for i in range(len(article_filenames)):
    # Read in the article
    with codecs.open(article_filenames[i], "r", encoding="utf8") as f:
      article = f.read()

    with open(label_filenames[i], "r") as f:
      reader = csv.reader(f, delimiter="\t")
      indices_list = []
      for row in reader:
        indices_list.append([int(row[1]), int(row[2])])

      # Merge the indices if overlapping
      indices_list = merge_overlapping(indices_list)

    # Add to the sequences
    sequences.append(article_labels_to_sequences(article, indices_list))

  # Concatenate all dataframes
  dataframe = pd.concat(sequences, ignore_index=True)

  return dataframe

In [7]:
def convert_dataframe_to_features(dataframe, max_seq_length, tokenizer):
  """
  Converts dataframe into features dataframe, where each feature will
  take form of [CLS] + A + [SEP]
  """
  # Create features
  features = pd.DataFrame(None, range(dataframe.shape[0]), 
                              ["input_ids", "input_mask", "segment_ids", "label_ids"])

  # For each sequence, do:
  for i in range(len(dataframe)):
    # Set first and second part of the sequences
    tokens = tokenizer.tokenize(dataframe["text"][i])

    # If length of the sequence is greater than max sequence length, truncate it
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    # Concatenate the tokens
    tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]

    # Compute the ids
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    pad_token = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
    input_ids = input_ids + [pad_token] * padding_length
    input_mask = input_mask + [0] * padding_length
    segment_ids = segment_ids + [0] * padding_length
    label_id = dataframe["label"][i]

    # Assert to make sure we have same length
    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    # Put the data into features dataframe
    features["input_ids"][i] = input_ids
    features["input_mask"][i] = input_mask
    features["segment_ids"][i] = segment_ids
    features["label_ids"][i] = label_id   

  return features

In [8]:
def generate_training_dataset_from_articles(articles_folders, labels_folders, tokenizer):
  """
  Generates dataset to go into BERT from articles and labels
  """
  
  # For each articles and labels folder set, turn them into dataframes
  dataframe_list = []
  for i in range(len(articles_folders)):
    dataframe_list.append(articles_to_dataframe(articles_folders[i], labels_folders[i]))

  # Concatenate the dataframes to make a total dataframe
  dataframe = pd.concat(dataframe_list, ignore_index=True)
  # Process into features dataframe
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset

#### Train

In [9]:
def train(train_dataset, model):
  """
  Trains the model with training dataset
  """
  # Initialize various necessary objects
  train_sampler = RandomSampler(train_dataset)
  train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
  # Compute the total time
  t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
  
  # Set the grouped parameters for optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
  
  # Compute warmup steps
  warmup_steps = math.ceil(t_total * args['warmup_ratio'])
  args['warmup_steps'] = warmup_steps if args['warmup_steps'] == 0 else args['warmup_steps']
  
  # Initialize optimizer as Adam with constant weight decay and a linear scheduler with warmup
  optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args['warmup_steps'], num_training_steps=t_total)

  # Initialize variables for training
  global_step = 0
  tr_loss = 0.0
  model.zero_grad()
  train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
  
  # Start training!
  for _ in train_iterator:
    epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      model.train()
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2], 
                'labels':         batch[3]}
      outputs = model(**inputs)
      loss = outputs[0]

      if args['gradient_accumulation_steps'] > 1:
        loss = loss / args['gradient_accumulation_steps']
          
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

      tr_loss += loss.item()
      if (step + 1) % args['gradient_accumulation_steps'] == 0:
        optimizer.step()
        scheduler.step() 
        model.zero_grad()
        global_step += 1

        if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
          output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
          if not os.path.exists(output_dir):
            os.makedirs(output_dir)
          model_to_save = model.module if hasattr(model, 'module') else model
          model_to_save.save_pretrained(output_dir)
          logger.info("Saving model checkpoint to %s", output_dir)

  return global_step, tr_loss / global_step

In [10]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args["model_type"]]
config = config_class.from_pretrained(args["model_name"], num_labels=2, finetuning_task="binary")
tokenizer = tokenizer_class.from_pretrained(args["model_name"])
model = model_class(config)
model.to(device)

train_dataset = generate_training_dataset_from_articles([train_articles], [train_SI_labels], tokenizer)
global_step, tr_loss = train(train_dataset, model)

model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(args['output_dir'])
tokenizer.save_pretrained(args['output_dir'])
torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))

INFO:LOG:Generating training dataset...
INFO:LOG:Generating dataframe for folder datasets/train
INFO:LOG:Creating features from dataframe


      label                                               text
0         0  буковинця засудили до довічного ув'язнення за ...
1         1  чоловік побив її, потім наніс ножові поранення...
2         0                                                 о.
3         0                                     друзі, у мене 
4         1                                              шален
...     ...                                                ...
3994      1  нехай краще 10 років ведуть переговори про мир...
3995      0  ", – сказав він. також у виступі на комітеті в...
3996      0                                                   
3997      1                повальна мобілізація теж не допомож
3998      0  е. навіть якщо призвемо зараз 4 млн українців ...

[3999 rows x 2 columns]
(3999, 2)


INFO:LOG:Creating TensorDataset from features dataframe
C:\Users\Ната\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]C:\Windows\Temp\ipykernel_1144\1346356067.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")


Iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 100%|██████████| 1/1 [03:34<00:00, 214.60s/it]
INFO:LOG: global_step = 500, average loss = 0.31195589937455953
INFO:LOG:Saving model checkpoint to output_model_SI


#### Evaluation

In [11]:
def generate_SI_eval_dataset_from_article(article_file, tokenizer):
  """
  Generates SI dataset to go into BERT from articles and labels
  """
  # Get the id name
  article_id = os.path.basename(article_file).split(".")[0][7:]

  # Read in the article
  with codecs.open(article_file, "r", encoding="utf8") as f:
    article = f.read()

  # For evaluating, do:
  dataframe = article_to_sequences(article_id, article, tokenizer)

  # Only include sequences that has alphanumeric characters in them
  bool_indices = [False] * len(dataframe)
  for i in range(len(dataframe)):
    if any(c.isalnum() for c in dataframe["text"][i]):
      bool_indices[i] = True
  dataframe = dataframe.loc[bool_indices].reset_index(drop=True)


  # Process into features dataframe
  features = convert_dataframe_to_features(dataframe, args['max_seq_length'], tokenizer)
      
  # Creating TensorDataset from features
  all_input_ids = torch.tensor(features["input_ids"], dtype=torch.long)
  all_input_mask = torch.tensor(features["input_mask"], dtype=torch.long)
  all_segment_ids = torch.tensor(features["segment_ids"], dtype=torch.long)
  all_label_ids = torch.tensor(features["label_ids"], dtype=torch.long)

  dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
  return dataset, dataframe


In [12]:
def classify_per_article(eval_dataframe, eval_dataset, model):
  """
  Classifies a single article dataset and returns article id with indices list
  """
  # Load the eval data and initialize sampler
  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

  # Start Classification
  preds = np.empty((0, 2)) 

  # For each batch, evaluate
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2],
                'labels':         batch[3]}
      outputs = model(**inputs)
      logits = outputs[1]

      preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

  # Get the most probable prediction
  preds = np.argmax(preds, axis=1)
  eval_dataframe["label"] = preds

  # Get the propaganda indices and return them
  eval_dataframe = eval_dataframe.loc[eval_dataframe["label"] ==1].reset_index(drop=True)
  indices_list = []
  for i in range(len(eval_dataframe)):
    indices_list.append(tuple([eval_dataframe["seq_starts"][i], eval_dataframe["seq_ends"][i]]))
  indices_list = merge_overlapping(indices_list)

  return eval_dataframe["id"][0], indices_list

In [13]:
article_filenames = sorted(glob.glob(os.path.join(dev_articles, "*.txt")))   

labels_dir = 'datasets/pred_SI_labels'
if not os.path.exists(labels_dir):
  os.makedirs(labels_dir)

for i in range(len(article_filenames)):
  eval_dataset, eval_dataframe = generate_SI_eval_dataset_from_article(article_filenames[i], tokenizer)
  article_id, indices_list = classify_per_article(eval_dataframe, eval_dataset, model, tokenizer)
  
  f = open(labels_dir + '/article{}.labels.tsv'.format(article_id), 'w')
  writer = csv.writer(f, delimiter='\t')
  for indices in indices_list:
    writer.writerow([article_id, indices[0], indices[1]])
  f.close()
  


INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


          id  seq_starts  seq_ends  label           text
0     477814           0         2      0             за
1     477814           0         4      0           запр
2     477814           0         6      0         запрош
3     477814           0        11      0    запрошувати
4     477814           0        13      0  запрошувати у
...      ...         ...       ...    ...            ...
5872  477814         328       335      0      ##ть ново
5873  477814         328       338      0   ##ть новость
5874  477814         331       335      0           ново
5875  477814         331       338      0        новость
5876  477814         335       338      0          ##сть

[5877 rows x 5 columns]
(5877, 5)


INFO:LOG:Creating TensorDataset from features dataframe
C:\Windows\Temp\ipykernel_1144\1158175120.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):


Evaluating:   0%|          | 0/735 [00:00<?, ?it/s]

INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


...

INFO:LOG:Generating evaluation dataset...
INFO:LOG:Creating features from dataframe


           id  seq_starts  seq_ends  label         text
0      996076           0         2      1           не
1      996076           0         5      1        не ви
2      996076           0         7      1      не викл
3      996076           0         8      1     не виклю
4      996076           0        11      1  не виключає
...       ...         ...       ...    ...          ...
13495  996076         464       470      1     ##писати
13496  996076         464       472      1   ##писатись
13497  996076         467       470      1        ##ати
13498  996076         467       472      1      ##атись
13499  996076         470       472      1         ##сь

[13500 rows x 5 columns]
          id  seq_starts  seq_ends  label                   text
0     996249           0         9      0              швидкість
1     996249           0        11      0            швидкість в
2     996249           0        13      0          швидкість віт
3     996249           0        15      0 

INFO:LOG:Creating TensorDataset from features dataframe
C:\Windows\Temp\ipykernel_1144\1158175120.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):


Evaluating:   0%|          | 0/833 [00:00<?, ?it/s]

          id  seq_starts  seq_ends  label                   text
0     996249           0         9      0              швидкість
1     996249           0        11      1            швидкість в
2     996249           0        13      1          швидкість віт
3     996249           0        15      1        швидкість вітру
4     996249           0        21      1  швидкість вітру понад
...      ...         ...       ...    ...                    ...
6654  996249         355       361      1               ##писати
6655  996249         355       363      1             ##писатись
6656  996249         358       361      1                  ##ати
6657  996249         358       363      1                ##атись
6658  996249         361       363      1                   ##сь

[6659 rows x 5 columns]


In [5]:
# Open the output file in write mode
labels_dir = 'datasets/pred_SI_labels'
with open("datasets/SI.labels_pred.txt", 'w', newline='') as f_total:
    writer = csv.writer(f_total, delimiter='\t')
    print(f_total)
    # Get a sorted list of .txt files in the specified directory
    labels_filenames = sorted(glob.glob(os.path.join(labels_dir, "*.labels.tsv")))
    print(labels_filenames)
    # Iterate through each file and write its content to the output file
    for filename in labels_filenames:
        with open(filename, 'r') as f:
            reader = csv.reader(f, delimiter='\t')
            for row in reader:
                writer.writerow(row)

<_io.TextIOWrapper name='datasets/SI.labels_pred.txt' mode='w' encoding='cp1251'>
['datasets/pred_SI_labels\\article1062476.labels.tsv', 'datasets/pred_SI_labels\\article1062480.labels.tsv', 'datasets/pred_SI_labels\\article1062575.labels.tsv', 'datasets/pred_SI_labels\\article1062722.labels.tsv', 'datasets/pred_SI_labels\\article1062744.labels.tsv', 'datasets/pred_SI_labels\\article1062751.labels.tsv', 'datasets/pred_SI_labels\\article1063058.labels.tsv', 'datasets/pred_SI_labels\\article1063082.labels.tsv', 'datasets/pred_SI_labels\\article1063177.labels.tsv', 'datasets/pred_SI_labels\\article1063217.labels.tsv', 'datasets/pred_SI_labels\\article1063289.labels.tsv', 'datasets/pred_SI_labels\\article1063460.labels.tsv', 'datasets/pred_SI_labels\\article1063475.labels.tsv', 'datasets/pred_SI_labels\\article1063526.labels.tsv', 'datasets/pred_SI_labels\\article1063603.labels.tsv', 'datasets/pred_SI_labels\\article1063878.labels.tsv', 'datasets/pred_SI_labels\\article1063931.labels.tsv',

#### References:
The structure of data preporation is the modification of Henry Kim's [implementation](https://medium.com/@jihwangk/fine-grained-propaganda-detection-and-classification-with-bert-dfad4acaa321).

The structure of train() function is a modification of Thilina Rajapakse’s [implementation](https://github.com/ThilinaRajapakse/pytorch-transformers-classification).